# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,282.49,80,80,1.79,NZ,1692948492
1,1,papatowai,-46.5619,169.4708,282.28,95,88,0.62,NZ,1692948521
2,2,edinburgh of the seven seas,-37.0676,-12.3116,287.46,97,100,7.92,SH,1692948489
3,3,wynyard,-41.0000,145.7167,283.67,63,13,4.81,AU,1692948553
4,4,puerto natales,-51.7236,-72.4875,271.55,92,92,0.66,CL,1692948553


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[(city_data_df['Max Temp'] >= 250) &
                               (city_data_df['Humidity'] >= 40) &
                               (city_data_df['Humidity'] <= 80) &
                               (city_data_df['Cloudiness'] <= 50) &
                               (city_data_df['Wind Speed'] <=  1.5)
                              ]

# Drop any rows with null values
vacation_df = vacation_df.dropna()

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,monkey bay,-14.0728,34.9201,297.81,49,13,1.00,MW,1692948561
34,34,mayahi,13.9553,7.6712,297.68,71,43,1.00,NE,1692948564
82,82,newman,37.3138,-121.0208,297.28,65,0,0.45,US,1692948581
106,106,karkira,36.9292,6.5856,302.97,42,0,0.35,DZ,1692948590
156,156,omis,43.4447,16.6886,303.25,43,0,0.98,HR,1692948614
188,188,dembi dolo,8.5333,34.8000,294.12,75,3,0.99,ET,1692948627
235,235,xapuri,-10.6517,-68.5044,296.53,57,0,1.48,BR,1692948646
267,267,fougamou,-1.2154,10.5838,300.89,65,43,1.07,GA,1692948659
283,283,kyabe,9.4515,18.9449,301.47,72,37,1.02,TD,1692948666
290,290,veinticinco de mayo,-35.4323,-60.1727,278.70,68,11,0.89,AR,1692948669


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df.copy()
hotel_df = hotel_df[['City', 'Lat', 'Lng', 'Humidity', 'Country']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
26,monkey bay,-14.0728,34.9201,49,MW,
34,mayahi,13.9553,7.6712,71,NE,
82,newman,37.3138,-121.0208,65,US,
106,karkira,36.9292,6.5856,42,DZ,
156,omis,43.4447,16.6886,43,HR,
188,dembi dolo,8.5333,34.8000,75,ET,
235,xapuri,-10.6517,-68.5044,57,BR,
267,fougamou,-1.2154,10.5838,65,GA,
283,kyabe,9.4515,18.9449,72,TD,
290,veinticinco de mayo,-35.4323,-60.1727,68,AR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 50000
   

params = {'apiKey': geoapify_key,
          'categories': 'accommodation.hotel',
             
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
       
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print(name_address.url)
    print(name_address.status_code)


    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?apiKey=6b864811746e47509894b850544cc18a&categories=accommodation.hotel&filter=circle%3A34.9201%2C-14.0728%2C50000&bias=proximity%3A34.9201%2C-14.0728
200
monkey bay - nearest hotel: Chimanga Resthouse
https://api.geoapify.com/v2/places?apiKey=6b864811746e47509894b850544cc18a&categories=accommodation.hotel&filter=circle%3A7.6712%2C13.9553%2C50000&bias=proximity%3A7.6712%2C13.9553
200
mayahi - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=6b864811746e47509894b850544cc18a&categories=accommodation.hotel&filter=circle%3A-121.0208%2C37.3138%2C50000&bias=proximity%3A-121.0208%2C37.3138
200
newman - nearest hotel: Hampton Inn & Suites
https://api.geoapify.com/v2/places?apiKey=6b864811746e47509894b850544cc18a&categories=accommodation.hotel&filter=circle%3A6.5856%2C36.9292%2C50000&bias=proximity%3A6.5856%2C36.9292
200
karkira - nearest hotel: استراحة دريم هاوس
https://api.geoapify.com/v2/places?apiKey=6b864811746e

,City,Lat,Lng,Humidity,Country,Hotel Name
26,monkey bay,-14.0728,34.9201,49,MW,Chimanga Resthouse
34,mayahi,13.9553,7.6712,71,NE,No hotel found
82,newman,37.3138,-121.0208,65,US,Hampton Inn & Suites
106,karkira,36.9292,6.5856,42,DZ,استراحة دريم هاوس
156,omis,43.4447,16.6886,43,HR,Hotel Plaža
188,dembi dolo,8.5333,34.8000,75,ET,No hotel found
235,xapuri,-10.6517,-68.5044,57,BR,Pousada Chapurys
267,fougamou,-1.2154,10.5838,65,GA,Auberge
283,kyabe,9.4515,18.9449,72,TD,No hotel found
290,veinticinco de mayo,-35.4323,-60.1727,68,AR,Jorly


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "Country",
    hover_cols = ['City', 'Country', 'Hotel Name']
    
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City,Hotel Name)